In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [11]:
#### green taxi: feature selection

os.chdir("/Users/whiplash/SJSU/Semester 2/Github/Predicting-High-Taxi-Demand-Regions/Data")
# green_raw = []
green_raw = pd.read_csv("green_tripdata_2014-07.csv")

In [12]:
green_raw.columns=[c.replace(" ", "") for c in green_raw.columns]  
green_raw['lpep_pickup_datetime']=green_raw['VendorID'] # the first 3 columns are mislabeled
green_raw.index=np.arange(green_raw.shape[0])  # index is wrong
green=green_raw[['lpep_pickup_datetime', 'Pickup_longitude', 'Pickup_latitude']]
green.columns=['Date', 'Lon', 'Lat']

In [13]:
green[0:5]

,Date,Lon,Lat
0,2014-07-01 22:33:05,-73.952805,40.799091
1,2014-07-01 22:12:19,-73.925301,40.806042
2,2014-07-01 10:33:40,-73.931709,40.797146
3,2014-07-01 08:54:25,-73.915695,40.772076
4,2014-07-01 08:42:46,-73.910339,40.744736


In [14]:
green["Lat"] = green["Lat"].astype(str)
green["Lon"] = green["Lon"].astype(str)
green["Coord"] = green[["Lat", "Lon"]].apply(lambda x: ",".join(x), axis=1)

/Users/whiplash/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/whiplash/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/whiplash/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [15]:
green[0:5]

,Date,Lon,Lat,Coord
0,2014-07-01 22:33:05,-73.95280456542969,40.79909133911133,"40.79909133911133,-73.95280456542969"
1,2014-07-01 22:12:19,-73.92530059814453,40.8060417175293,"40.8060417175293,-73.92530059814453"
2,2014-07-01 10:33:40,-73.93170928955078,40.79714584350585,"40.79714584350585,-73.93170928955078"
3,2014-07-01 08:54:25,-73.91569519042969,40.77207565307617,"40.77207565307617,-73.91569519042969"
4,2014-07-01 08:42:46,-73.91033935546875,40.74473571777344,"40.74473571777344,-73.91033935546875"


In [16]:
from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()
zipcode = {}
output=[]
# green["Zipcode"] = "NaN"
# for n in range(len(green)):
#     res = search.by_coordinate(float(green["Lat"][n]), float(green["Lon"][n]), radius=30, returns=1)
#     if len(res)>0:
#         green["Zipcode"][n] = str(res[0]["Zipcode"])


for n in range(len(green)):
    res = search.by_coordinate(float(green["Lat"][n]), float(green["Lon"][n]), radius=5, returns=1)
    if len(res)>0:
        output.append(str(res[0]["Zipcode"]))        
    else:
        output.append("NaN")

In [17]:
print(len(output))

1273973


In [18]:
green["Zipcode"] = output

/Users/whiplash/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
green[0:5]

,Date,Lon,Lat,Coord,Zipcode
0,2014-07-01 22:33:05,-73.95280456542969,40.79909133911133,"40.79909133911133,-73.95280456542969",10026
1,2014-07-01 22:12:19,-73.92530059814453,40.8060417175293,"40.8060417175293,-73.92530059814453",10454
2,2014-07-01 10:33:40,-73.93170928955078,40.79714584350585,"40.79714584350585,-73.93170928955078",10035
3,2014-07-01 08:54:25,-73.91569519042969,40.77207565307617,"40.77207565307617,-73.91569519042969",11103
4,2014-07-01 08:42:46,-73.91033935546875,40.74473571777344,"40.74473571777344,-73.91033935546875",11377


In [20]:
nanlist = []
for n in range(len(green)):
    if green["Zipcode"][n] == "NaN":
        nanlist.append(n)

In [21]:
len(nanlist)

1795

In [22]:
len(green)

1273973

In [23]:
green1 = green.drop(green.index[nanlist])

In [65]:
# zipcode = pd.read_csv("/Users/whiplash/SJSU/Semester 2/Github/Predicting-High-Taxi-Demand-Regions/Data/Zip_CodeNYC.csv")

In [71]:
# print(type(green["Zipcode"]))

<class 'pandas.core.series.Series'>


In [72]:
# print(type(zipcode["JURISDICTION NAME"]))

<class 'pandas.core.series.Series'>


In [24]:
date=pd.to_datetime(green1['Date'])
green1['Month']=date.dt.month
green1['Day']=date.dt.day
green1['Hour']=date.dt.hour
temp=pd.DatetimeIndex(date)
green1['Weekday']=temp.weekday
# data_model=data_final[['Month', 'Weekday', 'Hour']]

In [25]:
green1

,Date,Lon,Lat,Coord,Zipcode,Month,Day,Hour,Weekday
0,2014-07-01 22:33:05,-73.95280456542969,40.79909133911133,"40.79909133911133,-73.95280456542969",10026,7,1,22,1
1,2014-07-01 22:12:19,-73.92530059814453,40.8060417175293,"40.8060417175293,-73.92530059814453",10454,7,1,22,1
2,2014-07-01 10:33:40,-73.93170928955078,40.79714584350585,"40.79714584350585,-73.93170928955078",10035,7,1,10,1
3,2014-07-01 08:54:25,-73.91569519042969,40.77207565307617,"40.77207565307617,-73.91569519042969",11103,7,1,8,1
4,2014-07-01 08:42:46,-73.91033935546875,40.74473571777344,"40.74473571777344,-73.91033935546875",11377,7,1,8,1
5,2014-07-01 15:05:02,-73.9735336303711,40.76479721069336,"40.76479721069336,-73.9735336303711",10022,7,1,15,1
6,2014-07-01 14:50:30,-73.93948364257811,40.82086563110352,"40.82086563110352,-73.93948364257811",10030,7,1,14,1
7,2014-07-01 09:46:47,-73.9256134033203,40.82827377319336,"40.82827377319336,-73.9256134033203",10039,7,1,9,1
8,2014-07-01 14:11:02,-73.96800231933594,40.76219940185547,"40.76219940185547,-73.96800231933594",10022,7,1,14,1
9,2014-07-01 19:16:57,-73.73844909667969,40.66784286499024,"40.66784286499024,-73.73844909667969",11422,7,1,19,1


In [26]:
# group=green1.groupby(['Month', 'Day', 'Hour', 'Zipcode']).size().reset_index(name='count')

In [27]:
# group

In [28]:
green1.to_csv('nycmodeldatajuly.csv',encoding='utf-8')